# TLS (Transport Layer Security) Integration

## Contents
1. TLS Functions & Benefits
2. Requirements for TLS Integration
3. Creating TLS Certificates
4. TLS Server Implementation
5. TLS Client Implementation
6. Complete TLS Communication

## Prerequisites:

```bash
pip install cryptography
```

In [16]:
import socket
import ssl
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.backends import default_backend
import datetime
import threading
import time

## 1. TLS Functions & Benefits

**TLS (Transport Layer Security)** = Cryptographic protocol for secure network communication

### Core Functions:
1. **Encryption:** Protects data confidentiality (symmetric encryption: AES)
2. **Authentication:** Verifies server/client identity (certificates)
3. **Integrity:** Detects data tampering (HMAC/hash functions)

### Key Benefits:
- **Privacy:** Data encrypted in transit
- **Trust:** Certificate validation prevents MITM attacks
- **Compliance:** Required for GDPR, PCI-DSS, HIPAA
- **SEO:** Google ranks HTTPS sites higher
- **Universal:** Supported by all modern browsers/systems

### TLS Versions:
- TLS 1.0/1.1 - **Deprecated** (insecure)
- TLS 1.2 - Current standard
- TLS 1.3 - Latest (faster, more secure)

### Common Uses:
- HTTPS (web browsing)
- Email (SMTPS, IMAPS)
- VPN connections
- API communications
- Database connections

## 2. Requirements for TLS Integration

### Prerequisites:
1. **TLS Certificate** (X.509) - Server's public key + identity
2. **Private Key** - Corresponding private key for certificate
3. **CA Certificate** (optional) - For client verification
4. **SSL/TLS Library** - Python's `ssl` module

### Implementation Steps:
1. Generate/obtain certificate and private key
2. Create SSL context with security settings
3. Load certificate and private key into context
4. Wrap socket with SSL context
5. Perform TLS handshake
6. Communicate over encrypted connection

## 3. Creating TLS Certificates

**Using certificate from PKI challenge**

## 4. TLS Server Implementation

**Steps:**
1. Create TCP socket
2. Create SSL context
3. Load certificate and private key
4. Wrap socket with SSL
5. Accept TLS connections

In [17]:
def create_tls_server(host="127.0.0.1", port=1111):
    """
    TLS Server implementation
    """
    print(f"Starting TLS Server on {host}:{port}...\n")

    # Step 1: Create SSL context
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)

    # Step 2: Configure security settings
    context.minimum_version = ssl.TLSVersion.TLSv1_2

    # Step 3: Load certificate and private key
    context.load_cert_chain(
        certfile="../07_pki_certificates/certificate.crt",
        keyfile="../07_pki_certificates/private_key.key",
    )

    print("✓ SSL context created")
    print("✓ Certificate and key loaded")
    print(f"  Protocol: {context.protocol}")
    print(f"  Min TLS: {context.minimum_version}")
    print(f"  Max TLS: {context.maximum_version}")

    # Step 4: Create and bind socket
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind((host, port))
    server_socket.listen(5)

    print(f"\nServer listening on {host}:{port}")

    try:
        # Step 5: Accept connection and wrap with TLS
        client_socket, addr = server_socket.accept()
        print(f"\nConnection from {addr}")

        # Wrap socket with SSL
        tls_socket = context.wrap_socket(client_socket, server_side=True)
        print("TLS handshake completed")

        # Display connection info
        print("\nTLS Connection Info:")
        print(f"  Version: {tls_socket.version()}")
        print(f"  Cipher: {tls_socket.cipher()[0]}")
        print(f"  Bits: {tls_socket.cipher()[2]}")

        # Receive and send data
        data = tls_socket.recv(1024).decode("utf-8")
        print(f"\n✓ Received: {data}")

        response = "Hello from TLS Server! Your message was received securely."
        tls_socket.send(response.encode("utf-8"))
        print(f"Sent: {response}")

        tls_socket.close()

    except Exception as e:
        print(f"Server error: {e}")
    finally:
        server_socket.close()
        print("\nServer closed")


print("TLS Server function defined")

TLS Server function defined


## 5. TLS Client Implementation

**Steps:**
1. Create TCP socket
2. Create SSL context
3. Configure certificate verification
4. Wrap socket and connect
5. Verify server certificate

In [18]:
def create_tls_client(host="127.0.0.1", port=1111):
    """
    TLS Client implementation
    """
    print(f"Starting TLS Client connecting to {host}:{port}...\n")

    # Step 1: Create SSL context
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)

    # Step 2: Configure verification (disable for self-signed cert)
    context.check_hostname = False
    context.verify_mode = ssl.CERT_NONE  # For self-signed cert
    # For production with CA cert: context.verify_mode = ssl.CERT_REQUIRED
    # context.load_verify_locations('ca_cert.pem')

    print("✓ SSL context created")
    print(f"  Verify mode: {context.verify_mode}")
    print(f"  Check hostname: {context.check_hostname}")

    try:
        # Step 3: Create socket and wrap with SSL
        with socket.create_connection((host, port)) as sock:
            with context.wrap_socket(sock, server_hostname=host) as tls_socket:
                print("\n✓ Connected to server")
                print("✓ TLS handshake completed")

                # Display connection info
                print("\nTLS Connection Info:")
                print(f"  Version: {tls_socket.version()}")
                print(f"  Cipher: {tls_socket.cipher()[0]}")
                print(f"  Bits: {tls_socket.cipher()[2]}")

                # Get server certificate
                cert = tls_socket.getpeercert(binary_form=True)
                if cert:
                    cert_obj = x509.load_der_x509_certificate(cert, default_backend())
                    print("\nServer Certificate:")
                    print(f"  Subject: {cert_obj.subject.rfc4514_string()}")
                    print(f"  Issuer: {cert_obj.issuer.rfc4514_string()}")
                    print(
                        f"  Valid until: {cert_obj.not_valid_after.strftime('%Y-%m-%d %H:%M:%S')}"
                    )

                # Send and receive data
                message = "Hello from TLS Client!"
                tls_socket.send(message.encode("utf-8"))
                print(f"\n✓ Sent: {message}")

                response = tls_socket.recv(1024).decode("utf-8")
                print(f"✓ Received: {response}")

    except Exception as e:
        print(f"✗ Client error: {e}")

    print("\n✓ Client closed")


print("TLS Client function defined")

TLS Client function defined


## 6. Complete TLS Communication

**Running server and client together**

In [19]:
print("TLS CLIENT-SERVER COMMUNICATION DEMO")
print("=" * 70)

# Start server in separate thread
server_thread = threading.Thread(target=create_tls_server, daemon=True)
server_thread.start()

# Wait for server to start
time.sleep(1)

# Start client
print("\n" + "=" * 70)
print("CLIENT SIDE")
print("=" * 70)
create_tls_client()

# Wait for server thread to finish
server_thread.join(timeout=2)

TLS CLIENT-SERVER COMMUNICATION DEMO
Starting TLS Server on 127.0.0.1:1111...

✓ SSL context created
✓ Certificate and key loaded
  Protocol: 17
  Min TLS: 771
  Max TLS: -1

Server listening on 127.0.0.1:1111

CLIENT SIDE
Starting TLS Client connecting to 127.0.0.1:1111...

✓ SSL context created
  Verify mode: 0
  Check hostname: False

Connection from ('127.0.0.1', 55873)

✓ Connected to server
✓ TLS handshake completed

TLS Connection Info:
  Version: TLSv1.3
  Cipher: TLS_AES_256_GCM_SHA384
  Bits: 256

Server Certificate:
  Subject: CN=example.com,OU=IT Security,O=Awesome Dev Company,L=Wiener Neustadt,ST=Niederösterreich,C=AT
  Issuer: CN=example.com,OU=IT Security,O=Awesome Dev Company,L=Wiener Neustadt,ST=Niederösterreich,C=AT
  Valid until: 2026-11-08 10:56:14
TLS handshake completed

TLS Connection Info:
  Version: TLSv1.3
  Cipher: TLS_AES_256_GCM_SHA384
  Bits: 256

✓ Received: Hello from TLS Client!

✓ Sent: Hello from TLS Client!
Sent: Hello from TLS Server! Your message w

C:\Users\MarcelTurobin-Ort\AppData\Local\Temp\ipykernel_36840\471491114.py:41: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  f"  Valid until: {cert_obj.not_valid_after.strftime('%Y-%m-%d %H:%M:%S')}"


## 7. TLS Handshake Process

**What happens during TLS connection:**

1. Client Hello (Client sends)
     - Supported TLS versions
     - Cipher suites
     - Random number
     - Compression methods

2. Server Hello (Server responds)
     - Selected TLS version
     - Selected cipher suite
     - Random number
     - Session ID

3. Server Certificate (Server sends)
     - X.509 certificate (contains public key)
     - Certificate chain (if applicable)

4. Server Hello Done
   - Server indicates handshake messages complete

5. Client Certificate Verification (Client validates)
     - Certificate signature
     - Certificate validity period
     - Hostname match
     - Trust chain to CA

6. Client Key Exchange
     - Generates pre-master secret
     - Encrypts with server's public key
     - Sends to server

7. Master Secret Generation (Both sides)
     - Compute master secret from:
       - Pre-master secret
       - Client random
       - Server random
     - Derive session keys

8. Change Cipher Spec (Both sides)
     - Confirm switch to encrypted communication

9. Finished (Both sides)
     - Send encrypted hash of handshake
     - Verify integrity

10. Secure Communication
    - All data now encrypted with session keys
    - Symmetric encryption (AES) for speed
    - HMAC for integrity
  
![TLS Handshake](docs/tls_handshake.png)

## 9. Security Best Practices

### Server-Side:
1. Use TLS 1.2 minimum (TLS 1.3 preferred)
2. Strong cipher suites only
   - :white_check_mark: ECDHE-RSA-AES256-GCM-SHA384
   - :white_check_mark: ECDHE-RSA-AES128-GCM-SHA256
   - :x: RC4, 3DES, MD5 (weak)
3. Use 2048-bit RSA minimum (4096 preferred)
4. Enable Perfect Forward Secrecy (PFS)
5. Implement HSTS (HTTP Strict Transport Security)
6. Regular certificate renewal
7. Keep private keys secure (file permissions, HSM)

### Client-Side:
1. Always verify certificates (ssl.CERT_REQUIRED)
2. Validate hostname matches certificate
3. Check certificate expiration
4. Verify certificate chain to trusted CA
5. Implement certificate pinning for critical apps
6. Handle certificate errors properly

### Certificate Management:
1. Use CA-signed certificates in production
2. Self-signed only for development/testing
3. Automate renewal (Let's Encrypt)
4. Monitor expiration dates
5. Have revocation process
6. Use separate certs for different services

In [20]:
# Example: Secure SSL context configuration
print("\nSECURE SSL CONTEXT EXAMPLE:")
print("-" * 70)

context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)

# Minimum TLS version
context.minimum_version = ssl.TLSVersion.TLSv1_2

# Strong ciphers only
context.set_ciphers("ECDHE+AESGCM:ECDHE+CHACHA20:DHE+AESGCM:DHE+CHACHA20:!aNULL:!MD5:!DSS")

# Disable compression (CRIME attack)
context.options |= ssl.OP_NO_COMPRESSION

# Prefer server cipher order
context.options |= ssl.OP_CIPHER_SERVER_PREFERENCE

print("Secure context configured")
print(f"  Min version: {context.minimum_version}")
print(f"  Options: {hex(context.options)}")


SECURE SSL CONTEXT EXAMPLE:
----------------------------------------------------------------------
Secure context configured
  Min version: 771
  Options: 0x82520050


## 10. Testing TLS Connection

In [21]:
def test_tls_connection(host="localhost", port=443):
    """
    Test TLS connection and display security info
    """
    print(f"Testing TLS connection to {host}:{port}...\n")

    context = ssl.create_default_context()

    try:
        with socket.create_connection((host, port), timeout=5) as sock:
            with context.wrap_socket(sock, server_hostname=host) as ssock:
                print("✓ Connection successful\n")

                # TLS version
                print(f"TLS Version: {ssock.version()}")

                # Cipher info
                cipher = ssock.cipher()
                print(f"Cipher Suite: {cipher[0]}")
                print(f"Protocol: {cipher[1]}")
                print(f"Bits: {cipher[2]}")

                # Certificate info
                cert = ssock.getpeercert()
                print("\nCertificate Info:")
                print(f"  Subject: {dict(x[0] for x in cert['subject'])}")
                print(f"  Issuer: {dict(x[0] for x in cert['issuer'])}")
                print(f"  Valid from: {cert['notBefore']}")
                print(f"  Valid until: {cert['notAfter']}")

                if "subjectAltName" in cert:
                    print(f"  Alt Names: {[x[1] for x in cert['subjectAltName']]}")

                print("\n✓ TLS connection is secure")

    except ssl.SSLError as e:
        print(f"✗ SSL Error: {e}")
    except socket.timeout:
        print("✗ Connection timeout")
    except Exception as e:
        print(f"✗ Error: {e}")


print("TLS test function defined")
print("\nExample usage:")
print("  test_tls_connection('www.google.com', 443)")
print("  test_tls_connection('github.com', 443)")

TLS test function defined

Example usage:
  test_tls_connection('www.google.com', 443)
  test_tls_connection('github.com', 443)


## Summary

### TLS Functions:
1. **Encryption** - AES symmetric encryption for data
2. **Authentication** - X.509 certificates verify identity
3. **Integrity** - HMAC detects tampering

### Benefits:
- Protects data in transit
- Prevents MITM attacks
- Ensures privacy and trust
- Industry standard

### Implementation Requirements:
1. **Certificate** (X.509) with public key
2. **Private key** (must be kept secure)
3. **SSL context** with security settings
4. **TLS library** (Python ssl module)

### Implementation Steps:
1. Generate/obtain certificate and key
2. Create SSL context
3. Configure TLS version and ciphers
4. Load certificate and key
5. Wrap socket with SSL
6. Perform handshake
7. Communicate securely

### Best Practices:
- Use TLS 1.2+ (1.3 preferred)
- Strong cipher suites only
- Verify certificates in production
- Keep private keys secure
- Regular certificate renewal
- Implement HSTS

### Common Uses:
- HTTPS websites
- Email encryption
- VPN connections
- API communications
- Database connections